<a href="https://colab.research.google.com/github/yogasgm/data-science-machine-learning-BI/blob/main/Introduction_to_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Topic Modeling**

##**Importing required libraries**

In [ ]:
!pip install pyLDAvis --no-deps --quiet
!pip install funcy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import string
import gensim
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pyLDAvis.gensim_models
import warnings

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


##**Importing Dataset**

In [ ]:
# Fetching the dataset from GitHub
data_url = "https://raw.githubusercontent.com/andrybrew/IHT-SEM1302-30Okt/main/data/001_suku-bunga.csv"

# Using pandas read_csv function to load the data from the URL directly into a DataFrame
df_tweet = pd.read_csv(data_url)

##**Data Preprocessing for Topic Modeling**

In [ ]:
# Function for removing URL
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+|\S+\.\S+/\S+', '', text, flags=re.MULTILINE)

# Remove URL from each tweet
df_tweet['text'] = df_tweet['text'].apply(remove_url)

# Remove mentions entirely
df_tweet['text'] = df_tweet['text'].str.replace(r'@\S+', '', regex=True)

# Remove non-word characters except for spaces and %
df_tweet['text'] = df_tweet['text'].str.replace(r'[^\w\s%]', '', regex=True)

# Convert to lowercase
df_tweet['text'] = df_tweet['text'].str.lower()

# Trim leading and trailing spaces and replace multiple spaces with a single space
df_tweet['text'] = df_tweet['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [ ]:
# Select Tweets
tweet = df_tweet['text']

# Tokenize
tokenized_text = [d.lower().split() for d in tweet]

# Remove punctuation
punctuation = string.punctuation
tokenized_text = [[word for word in doc if word not in punctuation] for doc in tokenized_text]

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokenized_text = [[lemmatizer.lemmatize(word) for word in doc] for doc in tokenized_text]

# Remove stopwords
stop_words = stopwords.words('indonesian')
tokenized_text = [[word for word in doc if word not in stop_words] for doc in tokenized_text]

# Adding additional words to the stop words list
custom_stop_words = ['dgn', 'sdh', 'yg', 'the', 'gak', 'ga', 'a', 'krn', 'thd', 'nya', 'ya', 'n', 'kalo', 'aja', 'deh', 'tuh', 'udah', 'dll.', '2', '25', '20', '1.', '2.', '7.', 'u', '5', 'gua', '•']
stop_words.extend(custom_stop_words)

# Remove stopwords again
tokenized_text = [[word for word in doc if word not in stop_words] for doc in tokenized_text]

# Create dictionary
dictionary = corpora.Dictionary(tokenized_text)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in tokenized_text]

##**Setting Up LDA Model**

In [ ]:
# Train LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics= 3,
                                           passes = 22,
                                           per_word_topics=True,
                                           random_state=42)

##**Visualizing Topics**

In [ ]:
# Enable Notebook
pyLDAvis.enable_notebook()

# Visualize
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.130568 -0.008658       1        1  58.480369
2     -0.076083 -0.074187       2        1  23.739578
0     -0.054485  0.082846       3        1  17.780053, topic_info=         Term        Freq       Total Category  logprob  loglift
17      acuan  116.000000  116.000000  Default  30.0000  30.0000
19         bi  204.000000  204.000000  Default  29.0000  29.0000
24         6%   69.000000   69.000000  Default  28.0000  28.0000
50      turun   39.000000   39.000000  Default  27.0000  27.0000
295    persen   42.000000   42.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
244  investor    3.160078   11.843408   Topic3  -5.9695   0.4059
483    banget    2.944810    6.729019   Topic3  -6.0401   0.9007
124    kredit    3.033933   21.505088   Topic3  -6.0103  -0.2313
211    dampak    2.961503   12.124758   Topic3  -6.0344   0.3175
19         bi    3.043590  204.897352   Topic3  -6.0071  -2.4824

[203 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1554      2  0.825373     16
319       1  0.888465   2023
319       2  0.038629   2023
319       3  0.077258   2023
1064      1  0.974928    575
...     ...       ...    ...
16        2  0.575695  utang
583       2  0.778379   wall
583       3  0.311351   wall
1623      2  0.795449  warga
1623      3  0.198862  warga

[262 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [ ]:
# Generate the best topics
top_topics = lda_model.print_topics(num_words=10)  # Display the top 10 keywords for each topics

# Create DataFrame
df_topics = pd.DataFrame(top_topics, columns=['Topic', 'Keywords'])

# Set topic as index
df_topics.set_index('Topic', inplace=True)

# Show df_topics
df_topics

,Keywords
Topic,
0,"0.042*""bunga"" + 0.040*""suku"" + 0.011*""turun"" +..."
1,"0.080*""bunga"" + 0.077*""suku"" + 0.046*""bi"" + 0...."
2,"0.041*""bunga"" + 0.039*""suku"" + 0.009*""bank"" + ..."


**Topik 0:**

**Kata Kunci Utama:** bunga, suku, turun, kenaikan, fed, inflasi, saham, bank, masyarakat, ekonomi.

**Interpretasi:** Membahas dampak naik turunnya suku bunga terutama dari bank sentral (seperti Fed) terhadap ekonomi, pasar saham, dan inflasi.

**Topik 1:**

**Kata Kunci Utama:** bunga, suku, bi, acuan, 6%, rupiah, bank, indonesia, persen, naikkan.

**Interpretasi:** Fokus pada kebijakan suku bunga Bank Indonesia, efeknya pada nilai rupiah, dan ekonomi Indonesia.

**Topik 2:**

**Kata Kunci Utama:** bunga, suku, bank, fed, bi, turun, kenaikan, negara, inflasi, menaikkan.

**Interpretasi:** Berbicara tentang kebijakan suku bunga oleh bank-bank besar seperti Federal Reserve dan Bank Indonesia, serta efeknya terhadap inflasi dan ekonomi global dan nasional.